In [1]:
# https://github.com/evanm31/p4k-scraper/blob/master/data/scrapefork.py

In [2]:
# !python -m pip install retry
# !python -m pip install backoff

Import packages and set up enviornment

In [4]:
import urllib
import re
import time
import retry
import backoff
import pandas as pd
import requests
import datetime
import os
from bs4 import BeautifulSoup

import pitchfork_ratings_def # my custom functions from github

# If working on mac, use desktop folders, otherwise just work from git repo/ directory
if os.name == "posix":
    os.chdir("/Users/kylezengo/Desktop/DS/Music Ratings/")

Load historic data

In [7]:
# Load list of album review links
links_file_list = os.listdir("new_links")

links_list = []
for i in links_file_list:
    links = open("new_links/"+i)
    links = links.read().splitlines()
    
    links_list += links

links_list = list(set(links_list))
links_list = [x for x in links_list if x.startswith("https")]
print(f'{len(links_list)} album review links')

0 album review links


Get the link of each album review and save off to text file

In [9]:
%%time # 3-4s per page (5-10min to run)
new_links = []
i = 1
while requests.get("https://pitchfork.com/reviews/albums/?page="+str(i)).status_code==200:
    req = urllib.request.Request("https://pitchfork.com/reviews/albums/?page=" + str(i))
    html_page = pitchfork_ratings_def.urlopen_with_retry(req)
    print(f'{i}: html_page status code {html_page.getcode()}')

    soup = BeautifulSoup(html_page, "lxml")
    
    for link in soup.findAll('a'):
        link_get_href = link.get('href')
        if link_get_href == '/reviews/albums/':
            pass
        elif link_get_href.startswith( '/reviews/albums/?genre=' ):
            pass
        elif link_get_href.startswith( '/reviews/albums/' ):
            new_links.append("https://pitchfork.com"+link_get_href)
    i+=1

new_links = list(set(new_links))

1: html_page status code 200
2: html_page status code 200
3: html_page status code 200
4: html_page status code 200
5: html_page status code 200
6: html_page status code 200
7: html_page status code 200
8: html_page status code 200
9: html_page status code 200
10: html_page status code 200
11: html_page status code 200
12: html_page status code 200
13: html_page status code 200
14: html_page status code 200
15: html_page status code 200
16: html_page status code 200
17: html_page status code 200
18: html_page status code 200
19: html_page status code 200
20: html_page status code 200
21: html_page status code 200
22: html_page status code 200
23: html_page status code 200
24: html_page status code 200
25: html_page status code 200
26: html_page status code 200
27: html_page status code 200
28: html_page status code 200
29: html_page status code 200
30: html_page status code 200
31: html_page status code 200
32: html_page status code 200
33: html_page status code 200
34: html_page statu

In [10]:
len(new_links)

9982

In [11]:
# Save list since takes time to generate
with open(f'new_links/new_links_{datetime.datetime.now().date()}.txt', 'w') as f:
    for item in new_links:
        f.write("%s\n" % item)

Scrape reviews for each album and save to csv

In [13]:
# # test links
# new_links = ["https://pitchfork.com/reviews/albums/james-devane-searching/",
#              "https://pitchfork.com/reviews/albums/blood-incantation-absolute-elsewhere/",
#              "https://pitchfork.com/reviews/albums/body-meat-starchris/",
#              "https://pitchfork.com/reviews/albums/fred-again-actual-life-3-january-1-september-9-2022/",
#              "https://pitchfork.com/reviews/albums/100-gecs-10000-gecs/"]

In [14]:
%%time # 2s? per page (long time to run >1 hr)
dat = []
for idx, i in enumerate(new_links):
    print(f'{idx}: {i}')
    dat.append(pitchfork_ratings_def.gather_info(i))

new_reviews = pd.concat(dat).reset_index(drop=True)

0: https://pitchfork.com/reviews/albums/mary-margaret-ohara-miss-america/
1: https://pitchfork.com/reviews/albums/22493-adieux-au-dancefloor/
2: https://pitchfork.com/reviews/albums/caribou-honey/
3: https://pitchfork.com/reviews/albums/rza-ghost-dog-the-way-of-the-samurai/
4: https://pitchfork.com/reviews/albums/dorothy-carter-troubadour/
5: https://pitchfork.com/reviews/albums/richard-horowitz-eros-in-arabia/
6: https://pitchfork.com/reviews/albums/sexyy-red-in-sexyy-we-trust/
7: https://pitchfork.com/reviews/albums/blonde-redhead-sit-down-for-dinner/
8: https://pitchfork.com/reviews/albums/22425-alienist/
9: https://pitchfork.com/reviews/albums/dazy-outofbody/
10: https://pitchfork.com/reviews/albums/future-young-thug-super-slimey/
11: https://pitchfork.com/reviews/albums/washed-out-mister-mellow/
12: https://pitchfork.com/reviews/albums/six-organs-of-admittance-the-veiled-sea/
13: https://pitchfork.com/reviews/albums/fucked-up-one-day/
14: https://pitchfork.com/reviews/albums/provo

KeyboardInterrupt: 

In [18]:
new_reviews = pd.concat(dat).reset_index(drop=True)

In [15]:
# new_reviews

In [16]:
# # Testing errors
# album_link = "https://pitchfork.com/reviews/albums/various-artists-even-the-forest-hums-ukrainian-sonic-archives/" 
# gather_info(album_link)

# page = requests.get(album_link) #request URL
# soup = BeautifulSoup(page.content, 'html.parser') #parse with beautifulsoup
# soup

In [20]:
new_reviews.to_csv(f'new_reviews/new_reviews_{datetime.datetime.now().date()}.csv',index=False)